## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/weatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wagar,SD,16.1525,36.2032,100.20,16,9,22.46,clear sky
1,1,Norman Wells,CA,65.2820,-126.8329,42.84,52,40,14.97,scattered clouds
2,2,Nago,JP,26.5881,127.9761,66.45,80,100,9.93,light rain
3,3,Kiryat Gat,IL,31.6100,34.7642,68.40,100,2,4.03,clear sky
4,4,Hermanus,ZA,-34.4187,19.2345,62.80,84,0,10.74,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input('Please enter the minimum temperature for your trip: '))
max_temp = float(input('Please enter the maximum temperature for your trip: '))

Please enter the minimum temperature for your trip: 60
Please enter the maximum temperature for your trip: 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Nago,JP,26.5881,127.9761,66.45,80,100,9.93,light rain
3,3,Kiryat Gat,IL,31.6100,34.7642,68.40,100,2,4.03,clear sky
4,4,Hermanus,ZA,-34.4187,19.2345,62.80,84,0,10.74,clear sky
6,6,East London,ZA,-33.0153,27.9116,63.54,72,0,9.22,clear sky
7,7,Kapaa,US,22.0752,-159.3190,73.11,86,40,21.85,scattered clouds
8,8,Port Blair,IN,11.6667,92.7500,80.64,94,20,2.30,mist
12,12,Manaus,BR,-3.1019,-60.0250,86.50,78,40,4.61,scattered clouds
13,13,Albany,US,42.6001,-73.9662,65.57,31,13,1.01,few clouds
15,15,San Ramon,US,37.7799,-121.9780,63.28,66,0,4.61,clear sky
16,16,Basco,PH,20.4487,121.9702,78.67,81,96,22.75,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Nago,JP,66.45,light rain,26.5881,127.9761,
3,Kiryat Gat,IL,68.40,clear sky,31.6100,34.7642,
4,Hermanus,ZA,62.80,clear sky,-34.4187,19.2345,
6,East London,ZA,63.54,clear sky,-33.0153,27.9116,
7,Kapaa,US,73.11,scattered clouds,22.0752,-159.3190,
8,Port Blair,IN,80.64,mist,11.6667,92.7500,
12,Manaus,BR,86.50,scattered clouds,-3.1019,-60.0250,
13,Albany,US,65.57,few clouds,42.6001,-73.9662,
15,San Ramon,US,63.28,clear sky,37.7799,-121.9780,
16,Basco,PH,78.67,overcast clouds,20.4487,121.9702,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    params['location'] = f"{row['Lat']}, {row['Lng']}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        pass

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'weatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
    <dt>Hotel Name</dt>
        <dd>{Hotel Name}</dd>
    <dt>City</dt>
        <dd>{City}</dd>
    <dt>Country</dt>
        <dd>{Country}</dd>
    <dt>Current Weather</dt>
        <dd>{Current Description}</dd>
    <dt>Max Temp</dt>
        <dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))